<br><br><b><font size=10> CONTINUOUS CONTROL</font></b>
#### <i>...implementation for Udacity Deep Reinforcement Learning 
<hr><hr>

## Initialize Imports for the notebook
This Notebook uses code from separate python files where most of the implementation is handled

In [29]:
import environment as E
from buffers import ReplayBuffer
from agent import D4PG_Agent
import models
from logger import Logger
from progressbar import ProgressBar
#from get_args import get_args

import os.path
import time
import re
import sys
import importlib
import copy

import torch
import matplotlib.pyplot as plt
import numpy as np
from unityagents import UnityEnvironment
from collections import deque
import torchvision.transforms as T
import multiprocessing as multi
multi.cpu_count()

16

## Manually declare an ARGS class
<i> For testing code in the notebook, to take the place of argparser in the command line.

In [39]:
class Args:
    def __init__(self):
        self.train = True
        self.nographics = False
        self.num_eps = 10
        self.rollout = 5
        self.batchsize = 64
        self.pretrain = 1000
        self.num_episodes = 1
        self.max_time = 50
        
args = Args()

<b>Confirm that the args are all set the way we want them.

In [40]:
for arg in vars(args):
    if arg == "sep": continue
    print("{}: {}".format(arg.upper(), getattr(args, arg)))

TRAIN: True
NOGRAPHICS: False
NUM_EPS: 10
ROLLOUT: 5
BATCHSIZE: 64
PRETRAIN: 1000
NUM_EPISODES: 1
MAX_TIME: 50


## Load the environment
<i> 
And print a bit of information contained in the wrapper class
    
Set the training mode to FALSE while interactively learning about the code

In [4]:
env = E.Environment(args)
print("State size:", env.state_size)
print("Action size:", env.action_size)
print("Num Agents:", env.agent_count)
env.train = False


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


State size: 33
Action size: 4
Num Agents: 20


<hr>

# Test code as it's developed

<hr>

## Take random actions in the environment below 
<i>
-to check that code is working<br>
-to get familiar with the environment

In [7]:
env.reset()
scores = np.zeros(env.agent_count)
states = env.states
actions = np.zeros((20,4))
for i in range(10):
    actions = np.random.randn(env.agent_count, env.action_size)
    #actions[:,3] += .01
    actions = np.clip(actions, -1, 1)
    next_states, rewards, dones = env.step(actions)
    scores += rewards
    states = next_states
#     if np.any(dones):
#         break
#    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

Total score (averaged over agents) this episode: 0.09649999784305692


### Force-reload modules as they're updated
<i> This notebook was developed as the code is written in Atom, the below cell reloads the modules as they're needed.

In [19]:
import models
import agent
importlib.reload(models)
importlib.reload(agent)
from agent import D4PG_Agent

#### Load the AGENT!

In [20]:
d4pg_agent = D4PG_Agent(env.state_size, env.action_size, env.agent_count)
# print(d4pg_agent.__class__.__name__)
# print(d4pg_agent.memory)
# agent.initialize_memory(10, env)

In [13]:
env.reset()
states = env.states

### Test out Actor actions without training
<i> Test the <b>Actor</b> network

In [ ]:
env.reset()
scores = np.zeros(env.agent_count)
states = env.states
for i in range(30):
    actions = d4pg_agent.act(states)
    print("ACTIONS:", actions[1])
    next_states, rewards, dones = env.step(actions)
    scores += rewards
    states = next_states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

### Test out Critic scores without training
<i> Test the <b>Critic</b> network

In [ ]:
env.reset()
scores = np.zeros(env.agent_count)
states = env.states
for i in range(10):
    actions = d4pg_agent.act(states) 
    next_states, rewards, dones = env.step(actions)
    scores += rewards
    q, probs = d4pg_agent.critic(next_states, torch.from_numpy(actions))
    print(probs[0])
    print(q[0])
    #print(values.sample())
    states = next_states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

<hr><hr>

## Test out various Agent functionality

<hr><hr>

In [108]:
import models
import agent
import buffers
importlib.reload(models)
importlib.reload(agent)
importlib.reload(buffers)
from agent import D4PG_Agent
d4pg_agent = D4PG_Agent(env.state_size, env.action_size, env.agent_count)
env.train = True
env.reset()
d4pg_agent.initialize_memory(args.pretrain, env)

Initializing memory buffer.


In [97]:
print(len(d4pg_agent.memory))
print(d4pg_agent.memory.buffer[1])

1000
experience(states=tensor([[ 0.0000e+00, -4.0000e+00,  0.0000e+00,  1.0000e+00, -0.0000e+00,
         -0.0000e+00, -4.3711e-08,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0000e+01,
          0.0000e+00,  1.0000e+00, -0.0000e+00, -0.0000e+00, -4.3711e-08,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00, -7.7950e+00, -1.0000e+00, -1.7996e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00, -4.2133e-01]]), actions=tensor([[ 0.1974, -0.9684, -0.2740, -0.0876]], dtype=torch.float64), rewards=tensor([0.]), next_states=tensor([[-5.3564e-01, -3.8914e+00,  7.6649e-01,  9.9310e-01, -6.6460e-02,
          6.4298e-03,  9.6383e-02, -1.4801e+00, -2.5742e-02, -1.5480e-01,
         -6.2571e-01,  1.0494e+00,  5.8080e+00, -6.3519e-01, -9.5505e+00,
          1.3365e-01,  9.7623e-01,  4.1214e-02,  4.8610e-02, -2.0715e-01,
          3.1739e+00, -5.1678e-01, -3.4793e-01, -1.7483e+00,  6.5446e+00,
 

In [109]:
for episode in range(1, args.num_episodes+1):

    # Begin each episode with a clean environment
    env.reset()

    # Get initial state
    states = env.states

    # Gather experience for a maximum amount of steps, or until Done,
    # whichever comes first
    for t in range(args.max_time):
        actions = d4pg_agent.act(states)
        next_states, rewards, dones = env.step(actions)
        d4pg_agent.step(states, actions, rewards, next_states)
        states = next_states

        logger.rewards += rewards
        if np.any(dones):
            break

    d4pg_agent.reset_nstep_memory()

C:\Users\mattdoll\Development\deep-reinforcement-learning\p2_continuous-control\agent.py:201: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rewards = torch.tensor(rewards).unsqueeze(-1)


TypeError: expected sequence object with len >= 0 or a single integer

In [113]:
batch = d4pg_agent.memory.sample(64)
torch.cat(batch.state).shape

torch.Size([64, 33])

In [93]:
batch = d4pg_agent.memory.sample(64)
#torch.cat(batch.states)
batch.states[0]

tensor([ 1.9749, -2.2350, -2.6814,  0.8769,  0.2303,  0.1071, -0.4082,  1.6161,
        -0.3806,  0.3420,  1.8028,  4.8997, -3.0668,  0.9162, -5.0352, -3.5253,
         0.6781, -0.2834,  0.4757,  0.4832,  0.3891, -0.3861, -0.0654,  2.3597,
         8.1140, -7.4417, -0.3756, -1.0000,  7.9912,  0.0000,  1.0000,  0.0000,
         0.4962])

## Learn about how the Categorical Bellman step works

In [ ]:
actions = np.random.randn(env.agent_count, env.action_size)
actions = np.clip(actions, -1, 1).astype(np.float32)
states = torch.from_numpy(env.states).float()

In [ ]:
q, probs, log_probs = critic(states, torch.from_numpy(actions))

In [ ]:
print(probs.shape)
print(log_probs.shape)
print(q)

In [ ]:
rewards

In [ ]:
class Container():
    def __init__(self):
        pass
c = Container()
    

In [ ]:
c.vmin = -10
c.vmax = 10
c.natoms = 51
c.gamma = .99
c.atoms = torch.linspace(vmin, vmax, natoms)
c.delta_z = (vmax - vmin) / (natoms -1)
c.r = torch.tensor(rewards).unsqueeze(-1)

c.probs = probs.detach()
c.q_next = (probs * atoms).sum()


In [ ]:
def print_vars(x):
    for arg in vars(x):
        print("{}: {}".format(arg.upper(), getattr(x, arg)))

In [ ]:
c.atoms

### projected atoms
<html><i>
<b>atoms.view(1,-1)</b> becomes shape [1, num_atoms]
<br>
<b>r</b> is unsqueezed in the last (-1) dimension, so it's shape [20,1]
<br>
the result is a tensor that holds an offset (projected) version of the atoms for each reward instance

In [ ]:
np.set_printoptions(suppress=True)

#### tz = projected atoms, atoms (values) projected by scaling and offsetting via the bellman equation


In [ ]:
tz = c.r + c.gamma * c.atoms.view(1,-1)
tz.clamp_(vmin, vmax)

#### computes "bj" from the pseudo-code

In [ ]:
b = (tz - c.vmin) / c.delta_z
b[0].numpy()

#### l/u in the psuedocode are LOWER and UPPER bounds on the supports

In [ ]:
l = b.floor().long()
u = b.ceil().long()
print(l[0])
print(u[0])

#### m_l/m_u are computed in the pseudocode under "distribute the probability of tz", but still a bit opaque to me on how it should be used

In [ ]:
# version from GITHUB implementations

gdml = (u.float() + (l == u).float() - b) * c.probs
gdml[0].numpy()

In [ ]:
# version from GITHUB implementations

gdmu = (b - l.float()) * c.probs
gdmu[0].numpy()

In [ ]:
# version from the PAPER

pdml = l.float() + (c.probs * (u.float()-b))
pdml[0].numpy()

In [ ]:
# version from the PAPER

pdmu = u.float() + (c.probs * (b - l.float()))
pdmu[0].numpy()

In [ ]:
target_prob.size(0)

In [ ]:
c.probs[0].numpy()

In [ ]:
target_prob = torch.tensor(np.zeros(probs.size()))
for i in range(target_prob.size(0)):
    target_prob[i].index_add_(0, l[i].long(), gdml[i].double())
    target_prob[i].index_add_(0, u[i].long(), gdmu[i].double())
target_prob[0].numpy()

In [ ]:
# zip dml and dmu together offset by 1 index and subtract 1, receive amount to add to probs!
x = zip()

In [ ]:
for i in range(c.probs.size(1)):
    t = target_prob[0].numpy()[i]
    p = c.probs[0].numpy()[i]
    pad = 8
    print(str(t)[:pad], str(p)[:pad], str(t-p)[:pad])

In [ ]:
env.close()